In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
#from ta.volume import SMAEaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.trend import MACD
#from scipy.stats import percentileofscore
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:.2%}"
wind=14
for i in range(0,30):
    try:
        stk=sip[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()
        #print(stk)
        dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
        dfd['Date'] = pd.to_datetime(dfd.index)
        dfd['Week_Number'] = dfd['Date'].dt.isocalendar().week
        dfd['Year'] = dfd['Date'].dt.isocalendar().year
        dfd['Close1']=dfd['Close'].shift(1)
        dfd['Volume1']=dfd['Volume'].shift(1)
        dfd['Low1']=dfd['Low'].shift(1)
        dfd['High1']=dfd['High'].shift(1)
        dfd['Change']=dfd.Close-dfd.Close1
        dfd.set_index(['Year','Week_Number'],inplace=True)
        dfd['Mon']=dfd['Date'].dt.strftime('%m-%Y')


        dfw = dfd.groupby(['Year','Week_Number']).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
        dfw['Close1']=dfw['Close'].shift(1)
        dfw['Volume1']=dfw['Volume'].shift(1)
        dfw['Low1']=dfw['Low'].shift(1)
        dfw['High1']=dfw['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfw,how='left',left_index=True,right_index=True,suffixes=[None,'w'])
        #print(dfd.tail)
        dfm=dfd.resample('M', on='Date').agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum','Mon':'first'})
        dfm['Close1']=dfm['Close'].shift(1)
        dfm['Volume1']=dfm['Volume'].shift(1)
        dfm['Low1']=dfm['Low'].shift(1)
        dfm['High1']=dfm['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfm,how='left',left_on='Mon',right_on='Mon',suffixes=[None,'m'])
        dfd=dfd.dropna()


        dfd['fid']=ForceIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).force_index()
        dfd['fiw']=ForceIndexIndicator(close=dfd.Close1w,volume=dfd.Volume1w).force_index()
        dfd['fim']=ForceIndexIndicator(close=dfd.Close1m,volume=dfd.Volume1m).force_index()
        dfd['emd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).ease_of_movement()
        dfd['emw']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).ease_of_movement()
        dfd['emm']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).ease_of_movement()
        dfd['semd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).sma_ease_of_movement()
        dfd['semw']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).sma_ease_of_movement()
        dfd['semm']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).sma_ease_of_movement()
        dfd['mcd']=MACD(close=dfd.Close1).macd_diff()
        dfd['mcw']=MACD(close=dfd.Close1w).macd_diff()
        dfd['mcm']=MACD(close=dfd.Close1m).macd_diff()
        dfd['dlr']=DailyLogReturnIndicator(close=dfd.Close1).daily_log_return() 
        dfd=dfd.dropna()
        #print(dfd.head)
        x=1
        x2=1
        #print(dfd.columns)
        conditions = [ 
        (dfd['Open']>0)

        #& (dfd.fid>np.percentile(dfd.fid,100-x))
        #& (dfd.fiw>np.percentile(dfd.fiw,100-x))
        #& (dfd.fim>np.percentile(dfd.fim,100-x))
       # & (dfd.mcw>np.percentile(dfd.mcw,100-x))    
        #& (dfd.emd>np.percentile(dfd.emd,100-x))
        #& (dfd.emw>np.percentile(dfd.emw,100-x))
        #& (dfd.emm>np.percentile(dfd.emm,100-x))
        #& (dfd.semw>np.percentile(dfd.semw,100-x))
        #& (dfd.semm>np.percentile(dfd.semm,100-x))
        &(dfd.dlr>np.percentile(dfd.dlr,100-x))
        ,
        (dfd['Open']>0)
        #& (dfd.fid<np.percentile(dfd.fid,x))
        #& (dfd.fiw<np.percentile(dfd.fiw,x))
       # & (dfd.fim<np.percentile(dfd.fim,x))
        #& (dfd.mcw<np.percentile(dfd.mcw,x))
       # & (dfd.emd<np.percentile(dfd.emd,x))
        #& (dfd.emw<np.percentile(dfd.emw,x))
       # & (dfd.emm<np.percentile(dfd.emm,x))
        #& (dfd.semw<np.percentile(dfd.semw,x))
        #& (dfd.semm<np.percentile(dfd.semm,x))
        &(dfd.dlr<np.percentile(dfd.dlr,x))
        ]

        choices = [
        -1,1
        ]
        dfd['trade_pass']=np.select(conditions,choices,default=0)
        dfd_long=dfd[dfd['trade_pass']==1]
        dfd_short=dfd[dfd['trade_pass']==-1]
        #print(dfd_long)
        fir=dfd.iloc[0,0]

        las=dfd.iloc[len(dfd.index)-1,3]
        long_prof=(np.sum(dfd_long.Change)+fir)/fir-1
        short_prof=(-np.sum(dfd_short.Change)+fir)/fir-1
        trade_rate_long=len(dfd_long.index)/(len(dfd.index))
        trade_rate_short=len(dfd_short.index)/(len(dfd.index))
        hold_wr=len(dfd[dfd.Change>0])/(len(dfd.index))
        long_wr=len(dfd_long[dfd_long.Change>0])/(len(dfd_long.index))
        short_wr=len(dfd_short[dfd_short.Change<0])/(len(dfd_short.index))
        hold_prof=(las)/(fir)-1
        #print(hold)
        #print(long_prof)
        #print(trade_rate)
        #print(hold_wr)
        #print(trade_wr)
        #print(max(dfd.kd))
        temp=pd.Series([stk,hold_prof,long_prof,short_prof,hold_wr,long_wr,short_wr,trade_rate_long,trade_rate_short])
        dfout=dfout.append(temp,ignore_index=True)
    except:
        pass
print(dfout)
fin=[]
for j in range (1,len(dfout.columns)):
    temp=(np.average(dfout.iloc[:,j]))
    fin.append(temp)
print(fin)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********